# SQL query from table names - Continued

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("""bears, animals, refugee""", context_user))

```sql
CREATE TABLE bears (
    bear_id INTEGER PRIMARY KEY,
    name TEXT,
    species TEXT,
    age INTEGER
);

CREATE TABLE animals (
    animal_id INTEGER PRIMARY KEY,
    name TEXT,
    species TEXT,
    habitat TEXT
);

CREATE TABLE refugee (
    refugee_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    country_of_origin TEXT
);
```


In [13]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("bears, animals, refugee", old_context_user))

Something that can be solved with SQL.


In [11]:
#new
print(return_CCRMSQL("give me the bears which ate the most fish", context_user))

```sql
SELECT bear_name, SUM(fish_eaten) AS total_fish_eaten
FROM bears
GROUP BY bear_name
ORDER BY total_fish_eaten DESC;
```


In [12]:
#old
print(return_CCRMSQL("give me the bears which ate the most fish", old_context_user))

This is your SQL:
```sql
SELECT * 
FROM bears
WHERE favorite_food = 'fish'
ORDER BY amount_eaten DESC
LIMIT 1;
```

This SQL query selects the bear(s) from the table "bears" that have "fish" as their favorite food, orders them by the amount of fish eaten in descending order, and limits the result to only one row, giving you the bear that ate the most fish.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [14]:
# new
context_user = context.copy()
print(return_CCRMSQL("""cars, tracks, spaceships""", context_user))

```sql
CREATE TABLE cars (
    car_id INTEGER PRIMARY KEY,
    brand TEXT,
    model TEXT,
    year INTEGER,
    color TEXT
);

CREATE TABLE tracks (
    track_id INTEGER PRIMARY KEY,
    name TEXT,
    location TEXT,
    length REAL,
    difficulty TEXT
);

CREATE TABLE spaceships (
    spaceship_id INTEGER PRIMARY KEY,
    name TEXT,
    manufacturer TEXT,
    crew_size INTEGER,
    top_speed INTEGER
);
```


- Given 3 properties with no extra description.
- understands what a car, track and spaceship are and give related properties as columns (brand, model year , color, crew_size etc)

- Creates successfully and correctly 3 tables

In [15]:
#old
print(return_CCRMSQL("cars, tracks, spaceships", old_context_user))

Ask for a SQL-related question, please.


- Given 3 properties with no extra description. -> "Ask for a SQL-related question, please."

In [16]:
#old
print(return_CCRMSQL("create 3 tables with cars, tracks, spaceships", old_context_user))

This is your SQL:
```sql
CREATE TABLE cars (
    ID_car int,
    brand varchar,
    model varchar,
    year int
);

CREATE TABLE tracks (
    ID_track int,
    name varchar,
    location varchar,
    length float
);

CREATE TABLE spaceships (
    ID_spaceship int,
    name varchar,
    manufacturer varchar,
    crew_capacity int
);
```

This SQL code creates three tables: "cars" with columns for car ID, brand, model, and year; "tracks" with columns for track ID, name, location, and length; and "spaceships" with columns for spaceship ID, name, manufacturer, and crew capacity.


- Given bigger description: create 3 tables with cars, tracks, spaceships
- it creates the 3 tables with ids
- Correct
- nice column names as in the "new example"

In [18]:
#old
print(return_CCRMSQL("construct 3 tables with cars, tracks, spaceships with best practices", old_context_user))

This is your SQL:
```sql
CREATE TABLE cars (
    car_id INT PRIMARY KEY,
    brand VARCHAR(50),
    model VARCHAR(50),
    year INT
);

CREATE TABLE tracks (
    track_id INT PRIMARY KEY,
    name VARCHAR(50),
    location VARCHAR(100),
    length FLOAT
);

CREATE TABLE spaceships (
    spaceship_id INT PRIMARY KEY,
    name VARCHAR(50),
    manufacturer VARCHAR(50),
    crew_capacity INT
);
```

This SQL code creates three tables named `cars`, `tracks`, and `spaceships` with appropriate fields for each entity following best practices.


In [19]:
# new
context_user = context.copy()
print(return_CCRMSQL("construct 3 tables with cars, tracks, spaceships with best practices", context_user))

```sql
CREATE TABLE cars (
    id INTEGER PRIMARY KEY,
    brand TEXT,
    model TEXT,
    year INTEGER,
    color TEXT
);

CREATE TABLE tracks (
    id INTEGER PRIMARY KEY,
    name TEXT,
    location TEXT,
    length REAL,
    difficulty TEXT
);

CREATE TABLE spaceships (
    id INTEGER PRIMARY KEY,
    name TEXT,
    manufacturer TEXT,
    crew_capacity INTEGER,
    top_speed INTEGER
);
```


- Given to both old and new : construct 3 tables with cars, tracks, spaceships with best practices

| Aspect                 | 🆕 **New Result**                                                      | 🕰️ **Old Result**                                                   |
| :--------------------- | :--------------------------------------------------------------------- | :------------------------------------------------------------------- |
| **Output Style**       | Returns only SQL code                                                  | Returns SQL **plus explanatory text**                                |
| **Column Types**       | Uses generic SQLite-style types (`TEXT`, `INTEGER`, `REAL`)            | Uses SQL-standard types (`VARCHAR`, `INT`, `FLOAT`)                  |
| **Primary Key Naming** | Uses simple column names (`id`)                                        | Uses descriptive primary keys (`car_id`, `track_id`, `spaceship_id`) |
| **Additional Fields**  | Adds creative extra fields like `color`, `difficulty`, and `top_speed` | Keeps a minimal set of fields                                        |
| **Formatting**         | Clean SQL block only                                                   | Includes heading (“This is your SQL”) and descriptive paragraph      |


# 📝 Report: SQL Table Generation Experiments

## **Objective**
The goal of this exercise was to test different prompt variations to generate SQL tables (`cars`, `tracks`, `spaceships`) using GPT-based models. The task was to explore how prompt wording affects output, creativity, correctness, and adherence to best practices.

---

## **Experiments**

1. **Prompt 1:** `"cars, tracks, spaceships"`  
   - **Result:** Pure SQL output with extra fields (`color`, `difficulty`, `top_speed`, `crew_size`).  
   - **Observation:** Model added creative columns not explicitly requested. Primary keys varied between `id` and `car_id`. Column types were `TEXT`, `INTEGER`, `REAL`.  
   - **Issue:** Naming and type conventions were inconsistent.  

2. **Prompt 2:** `"construct 3 tables with cars, tracks, spaceships with best practices"`  
   - **Result:** Clean SQL output, slightly more consistent naming (`id` as primary key), included extra fields for creativity (`color`, `top_speed`).  
   - **Observation:** More aligned with “best practices” in structure, but still lacked standardized types (`VARCHAR`, `INT`).  
   - **Issue:** No explanatory text, which may be less readable for teaching or documentation.  

3. **Prompt 3 (Old Context):** `"create 3 tables with cars, tracks, spaceships"`  
   - **Result:** SQL + explanation text, minimal columns, standardized types (`INT`, `VARCHAR`, `FLOAT`).  
   - **Observation:** Follows strict naming conventions (`ID_car`, `ID_track`), no extra fields, includes descriptive guidance.  
   - **Issue:** Less creative, fewer extra features, and sometimes too rigid.  

---

## **Findings**

- **Variations that worked well:**  
  - Prompts that explicitly included “best practices” led to more consistent primary key naming and clearer structure.  
  - New prompts generated additional creative fields that could be useful in practice.  

- **Variations that didn’t work well:**  
  - Some outputs introduced unnecessary or inconsistent naming (`id` vs `car_id`) — small hallucinations.  
  - Old prompts sometimes returned explanations instead of pure SQL, which is not ideal if only executable SQL is needed.  

- **Hallucinations observed:**  
  - Adding columns not mentioned in the prompt (e.g., `color`, `difficulty`, `top_speed`).  
  - Sometimes mixing field names slightly differently than best practices.  

---

## **Learnings**

1. **Prompt design matters** — slight changes in wording strongly influence the model’s output style and content.  
2. **Creativity vs. correctness trade-off** — new prompts allow extra fields and creative structure, but may slightly deviate from strict SQL conventions.  
3. **Context awareness** — old context models produced more formal and documented SQL, useful for teaching but less creative.  
4. **Checking outputs** — always review generated SQL for correctness, naming consistency, and alignment with intended database standards.  

---

## **Conclusion**
Different prompt variations produce significantly different SQL outputs. Careful prompt design allows balancing **creativity**, **best practices**, and **readability/executability**. Some hallucinations are normal, so outputs should always be validated before production use.
